# Building GPT from scratch

## Import the dataset and stuff

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-05-10 14:12:42--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  4.44MB/s    in 0.2s    

2025-05-10 14:12:43 (4.44 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
with open("input.txt","r",encoding='utf-8') as file :
    text = file.read()

In [4]:
print("length of dataset in characters : ",len(text))

length of dataset in characters :  1115394


In [5]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


## Starting up codes

In [6]:
# here all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# create a mapping from characters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i : ch for i,ch in enumerate(chars)}
encode = lambda s : [stoi[c] for c in s]    # encoder -- take a string, output a list of integers
decode = lambda l : ''.join(itos[i] for i in l)   # decoder -- take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:30])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43])


In [9]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8   # umnuhiig boduul, ehnii character-aas l ehleed daraagiinhiiga predict hiigeed yavna.
train_data[:block_size +1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context}, the target : {target}")

When input is tensor([18]), the target : 47
When input is tensor([18, 47]), the target : 56
When input is tensor([18, 47, 56]), the target : 57
When input is tensor([18, 47, 56, 57]), the target : 58
When input is tensor([18, 47, 56, 57, 58]), the target : 1
When input is tensor([18, 47, 56, 57, 58,  1]), the target : 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), the target : 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target : 58


In [12]:
# hyperparameters

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337);

n_embd = 384
batch_size = 64
block_size = 256  # what is the maximum context length for predictions?
max_iters = 5001
eval_interval = 500
learning_rate = 3e-4
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
n_layer = 6
dropout = 0.2
n_head = 6

In [14]:
torch.manual_seed(1337)
bo_size = 8
ba_size = 4
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - bo_size,(ba_size,))
    x = torch.stack([data[i:i+bo_size] for i in ix])
    y = torch.stack([data[i+1 : i+bo_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

xb,yb = get_batch("train")
print("Inputs : ")
print(xb.shape)
print(xb)
print("targets : ")
print(yb.shape)
print(yb)

print("-------")
for b in range(1):
    for t in range(bo_size):  # batch_size gej bas har.
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target : {target}")

Inputs : 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets : 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-------
when input is [24] the target : 43
when input is [24, 43] the target : 58
when input is [24, 43, 58] the target : 5
when input is [24, 43, 58, 5] the target : 57
when input is [24, 43, 58, 5, 57] the target : 1
when input is [24, 43, 58, 5, 57, 1] the target : 46
when input is [24, 43, 58, 5, 57, 1, 46] the target : 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target : 39


In [17]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [18]:
x,y = get_batch("train")

In [19]:
x.shape , y.shape

(torch.Size([64, 256]), torch.Size([64, 256]))

In [15]:
# estimate loss function
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train","val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

### The Mathematical trick in self-attention

### Trying masked self-attention

In [16]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a,1,keepdim=True)  # 3x1 size-tai row bolgon ni ter row-iinho sum-iig shaasan pisda baina.
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print("a = ")
print(a)
print("b = ")
print(b)
print("c = ")
print(c)

a = 
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b = 
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c = 
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 32])

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)
print(xbow.shape)
print(xprev.shape)

torch.Size([4, 8, 32])
torch.Size([8, 32])


In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
xbow2.shape

torch.Size([4, 8, 32])

In [ ]:
# version 3 : use Softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0 , float("-inf"))
wei = F.softmax(wei,dim=-1)
xbow3 = wei @ x
torch.allclose(xbow2,xbow3)

True

In [ ]:
# version 4 : self-attention
torch.manual_seed(1337)
B,T,C = 4,8,32   # batch_size , time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size,bias=False)

k = key(x)    # (B,T,16)
q = query(x)  # (B,T,16)

wei = q @ k.transpose(-2,-1)   # (B,T,16)  @ (B,16,T) ---> (B,T,T)


tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril==0, float("-inf"))
wei = F.softmax(wei,dim=-1)

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

## Self Attention Head

In [20]:
class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self,head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size,bias=False)   # (C,h) -> (384,64)
        self.query = nn.Linear(n_embd,head_size,bias=False)
        self.value = nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))   # (256,256)

        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        B,T,C = x.shape  # (64, 256, 384)
        k = self.key(x)  # → (64, 256, 64)  -> (B,T,h)
        q = self.query(x)
        v = self.value(x)

        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5  #  (B,T,h) @ (B,h,T) ---> (B,T,T)
        wei = wei.masked_fill(self.tril[:T,:T]==0,float("-inf"))
        wei = F.softmax(wei,dim=-1)
        wei = self.dropout(wei)
        #perform the weighted aggregation of the values
        out = wei @ v  # (B,T,T) @ (B,T,h) --->  (B,T,h)
        return out    # → (64, 256, 64)

In [28]:
value = nn.Linear(n_embd,64,bias=False)
x = torch.randn((64,256,384))
value(x).shape   # each head outputs 64, and it will be concataned back to 384 . cuz of 6 multi-heads

torch.Size([64, 256, 64])

## Multi Head Attention

In [ ]:
class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""
    def __init__(self,num_heads,head_size):   # num_heads = 6 , head_size = 64
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size,n_embd)

    def forward(self,x):
        out =  torch.cat([h(x) for h in self.heads],dim=-1)  #   6 * (64, 256, 64)  ---> (64, 256, 384)
        out = self.proj(out)
        return out   # -----> (64, 256, 384)

## Feed Forward Network

In [ ]:
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity"""
    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd,4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd,n_embd),
            nn.Dropout(dropout)
        )
    def forward(self,x):
        return self.net(x)

## Block

In [ ]:
class Block(nn.Module):
    """ Transformer block : communication followed by computation """
    def __init__(self,n_embd,n_head):
        # n_embd : embedding dimension, n_head : the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head,head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self,x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x    #  ---> (64, 256, 384)

## BigramLanguageModel

In [ ]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size,n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd,vocab_size)

    def forward(self, idx, targets=None):
        B,T = idx.shape   # (64,256)

        # idx and targets are both (B,T) tensor of integers
        tok_embd = self.token_embedding_table(idx)   # (B,T,C)   # (64,265,384)
        pos_embd = self.position_embedding_table(torch.arange(T,device=device))  # (T,C)   # ingej loop hiij, vector-uudaa index-lej avjgaa genee. ternees busdaar bol, shineer uusgeegui.
        x = tok_embd + pos_embd   # (B,T,C)
        x = self.blocks(x)    # (B,T,C)   # (64,256,384)
        x = self.ln_f(x)
        logits = self.lm_head(x)   # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape    # (64,256,65)
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to the last block_size tokens
            idx_cond = idx[:,-block_size:]
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx


In [ ]:
model = BigramLanguageModel()
m = model.to(device)

logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))
# # input just 1 number which is 0. and generate 100 chars. them pluck out the numbers from tensor,change it to list for decoding it.


torch.Size([16384, 65])
tensor(4.3299, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

## Training

In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on the train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb,yb = get_batch("train")

    # evaluate the loss
    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 4.3246, val loss 4.3234
step 500: train loss 1.6636, val loss 1.8272
step 1000: train loss 1.3348, val loss 1.5689
step 1500: train loss 1.2071, val loss 1.5049
step 2000: train loss 1.1152, val loss 1.4949
step 2500: train loss 1.0300, val loss 1.5125
step 3000: train loss 0.9434, val loss 1.5635
step 3500: train loss 0.8519, val loss 1.6224
step 4000: train loss 0.7587, val loss 1.7002
step 4500: train loss 0.6730, val loss 1.7825
step 5000: train loss 0.5891, val loss 1.8790


# Fullll Codeee !!!

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)



0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5090, val loss 2.5058
step 300: train loss 2.4197, val loss 2.4337
step 400: train loss 2.3501, val loss 2.3562
step 500: train loss 2.2965, val loss 2.3127
step 600: train loss 2.2412, val loss 2.2502
step 700: train loss 2.2053, val loss 2.2196
step 800: train loss 2.1631, val loss 2.1862
step 900: train loss 2.1233, val loss 2.1499
step 1000: train loss 2.1035, val loss 2.1306
step 1100: train loss 2.0688, val loss 2.1182
step 1200: train loss 2.0383, val loss 2.0801
step 1300: train loss 2.0240, val loss 2.0645
step 1400: train loss 1.9929, val loss 2.0366
step 1500: train loss 1.9699, val loss 2.0311
step 1600: train loss 1.9599, val loss 2.0447
step 1700: train loss 1.9396, val loss 2.0123
step 1800: train loss 1.9080, val loss 1.9932
step 1900: train loss 1.9080, val loss 1.9860
step 2000: train loss 1.8818, val loss 1.9941
step 2100: train loss 1.

In [ ]:

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



step 0: train loss 1.5164, val loss 1.6936
step 100: train loss 1.5227, val loss 1.6986
step 200: train loss 1.5153, val loss 1.7047
step 300: train loss 1.5221, val loss 1.7085
step 400: train loss 1.5113, val loss 1.6979
step 500: train loss 1.5129, val loss 1.7072
step 600: train loss 1.5062, val loss 1.7093
step 700: train loss 1.5101, val loss 1.6932
step 800: train loss 1.5126, val loss 1.7024
step 900: train loss 1.5242, val loss 1.7203
step 1000: train loss 1.4974, val loss 1.7110
step 1100: train loss 1.5074, val loss 1.7045
step 1200: train loss 1.5117, val loss 1.6946
step 1300: train loss 1.5025, val loss 1.6910
step 1400: train loss 1.5103, val loss 1.7055
step 1500: train loss 1.5055, val loss 1.7292
step 1600: train loss 1.5063, val loss 1.7018
step 1700: train loss 1.5224, val loss 1.6941
step 1800: train loss 1.5107, val loss 1.7131
step 1900: train loss 1.5021, val loss 1.6824
step 2000: train loss 1.5035, val loss 1.7135
step 2100: train loss 1.5117, val loss 1.7082


In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))



PAULINA:
Unto wint. My lord is noble: heaven Warwick to o' the shall now than the worlsel:
These most reasoned maid from know,
And is a boughness sigh murders:
Where is drumst to his words.

ANGELO:
Say, my name eye say soes
It is dilip amons, will go have: I do, he dagry,
That bholds him the fastly but of two desire to love,
His help.

NORFOLK:
Say, sir, likecing of his kell or unlayable
And Citizen:
Where, like him
Against the sile him: it is no gentled
Being his redressess.

DUKE VINCENTIO:
O, my man is?
You more on times about the villain
Was in her O.

OXFORD:
Why, dare my smily bring soul, thou well.

MENENIUS:
Sirry:
Less brother but sons her well-king, but
to marry consento thun of kning;
With, gently and you will to his queen will,
In all pagain it, and askeUngel a was to
to him, I have majesty talk! Was more I'll uselish in place have boy!
On shall near weapen; thou, men
your fathio, on and in rapnised,
Whene'er is, sir, Catesby, and shing yet,
With a witdness to part of al